In [12]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name
        
    # Функция-геттер
    @property
    def first_name(self):
        return self._first_name
    
    # Функция-сеттер
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value
        
    # Функция-делитер (необязательная)
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute")
    

a = Person('Guido')
print(a.first_name) # Вызывает геттер
a.first_name = '42' # Вызывает сеттер
print(a.first_name)

Guido
42


In [8]:
import math

class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    @property
    def area(self):
        return math.pi * self.radius ** 2
    
    @property
    def perimeter(self):
        return 2 * math.pi * self.radius
    

c = Circle(4.0)
print(c.radius)
print(c.area)
print(c.perimeter)

4.0
50.26548245743669
25.132741228718345


In [19]:
class SubPerson(Person):
    @property
    def first_name(self):
        print('Getting name')
        return super().first_name 
    
    @first_name.setter
    def first_name(self, value):
        print('Setting name to', value)
        super(SubPerson, SubPerson).first_name.__set__(self, value)
        
    @first_name.deleter
    def first_name(self):
        print('Deleting name')
        super(SubPerson, SubPerson).first_name.__delete__(self)
        
s = SubPerson('Guido')
print(s)
print(s.first_name)
s.first_name = 'Larry'
print(s.first_name)


Setting name to Guido
Getting name
Guido
Setting name to Larry
Getting name
Larry


In [21]:
# Дескриптор атрибута для целочисленного атрибута с проверкой типа
class Integer:
    def __init__(self, name):
        self.name = name

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('Expected an int')
        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        del instance.__dict__[self.name]
        

class Point:
    x = Integer('x')
    y = Integer('y')
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
p = Point(2, 3)
print(p.x) # Вызывает Point.x.__get__(p,Point)
p.y = 5 # Вызывает Point.y.__set__(p, 5)
print(p.y)

2
5


In [26]:
class lazyproperty:
    def __init__(self, func):
        self.func = func
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value
        

class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2
    
    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius
    

c = Circle(4.0)
print(c.radius)
print(c.area)
print(c.area)
print(c.perimeter)
print(c.perimeter)

# Получение переменных экземпляра
print(vars(c))

4.0
Computing area
50.26548245743669
50.26548245743669
Computing perimeter
25.132741228718345
25.132741228718345
{'radius': 4.0, 'area': 50.26548245743669, 'perimeter': 25.132741228718345}


In [31]:
class Structure:
    # Переменная класса, которая определяет ожидаемые поля
    _fields= []
    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError(f'Expected {len(self._fields)} arguments')
        
        # Устанавливает аргументы
        for name, value in zip(self._fields, args):
            setattr(self, name, value)
            
    
# Пример определения класса
class Stock(Structure):
    _fields = ['name', 'shares', 'price']
    
class Point(Structure):
    _fields = ['x','y']
    
class Circle(Structure):
    _fields = ['radius']
    def area(self):
        return math.pi * self.radius ** 2
    
s = Stock('ACME', 50, 91.1)
p = Point(2, 3)
c = Circle(4.5)

print(s)
print(p)
print(c.area())

63.61725123519331


In [35]:
import io
from abc import ABCMeta, abstractmethod

class IStream(metaclass=ABCMeta):
    @abstractmethod
    def read(self, maxbytes=-1):
        pass
    
    @abstractmethod
    def write(self, data):
        pass
    
def serialize(obj, stream):
    if not isinstance(stream, IStream):
        raise TypeError('Expected an IStream')
    
# Регистрируем встроенные классы ввода-вывода
# в качестве поддерживающих для нашего интерфейса
IStream.register(io.IOBase)

# Открыть обычный файл и провести проверку типа
with open('foo.txt') as f:
    print(isinstance(f, IStream))

True


In [42]:
# Базовый класс. Использует дескриптор для установки значения
class Descriptor:
    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)
            
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value
        
# Дескриптор для принудительного определения типов
class Typed(Descriptor):
    expected_type = type(None)
    
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('expected ' + str(self.expected_type))
        super().__set__(instance, value)
        
# Дескриптор для принудительного определения значений
class Unsigned(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super().__set__(instance, value)
        
class MaxSized(Descriptor):
    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
        super().__init__(name, **opts)
        
    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super().__set__(instance, value)
        

class Integer(Typed):
    expected_type = int

class UnsignedInteger(Integer, Unsigned):
    pass

class Float(Typed):
    expected_type = float

class UnsignedFloat(Float, Unsigned):
    pass

class String(Typed):
    expected_type = str

class SizedString(String, MaxSized):
    pass


class Stock:
    # Определяем ограничения
    name = SizedString('name', size=8)
    shares = UnsignedInteger('shares')
    price = UnsignedFloat('price')
    
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price
        
        
s = Stock('ACME', 50, 91.1)
print(s.name)

s.shares = 10 
print(s.shares)       
        

ACME
10


In [50]:
import bisect
from collections.abc import Sequence

class SortedItems(Sequence):
    def __init__(self, initial=None):
        self._items = sorted(initial) if initial is not None else []

    # Требуемые методы последовательности
    def __getitem__(self, index):
        return self._items[index]

    def __len__(self):
        return len(self._items)

    # Метод для добавления элемента в правильное место
    def add(self, item):
        bisect.insort(self._items, item)
        

items = SortedItems([5, 1, 3])
print(list(items))
items.add(2)
print(list(items))

[1, 3, 5]
[1, 2, 3, 5]


In [58]:
from collections.abc import MutableSequence

class Items(MutableSequence):
    def __init__(self, initial=None):
        self._items = list(initial) if initial is not None else []

    # Обязательные методы последовательности
    def __getitem__(self, index):
        print('Getting:', index)
        return self._items[index]
    
    def __setitem__(self, index, value):
        print('Setting:', index, value)
        self._items[index] = value
    
    def __delitem__(self, index):
        print('Deleting:', index)
        del self._items[index]
    
    def insert(self, index, value):
        print('Inserting:', index, value)
        self._items.insert(index, value)
    
    def __len__(self):
        print('Len')
        return len(self._items)
    
a = Items([1, 2, 3])
print(len(a))
a.append(4)
a.count(2)
a.remove(3)

Len
3
Len
Inserting: 3 4
Getting: 0
Getting: 1
Getting: 2
Getting: 3
Getting: 4
Getting: 0
Getting: 1
Getting: 2
Deleting: 2


In [63]:
class A:
    def spam(self, x):
        print("Spam:", x)
    
    def foo(self):
        pass

class B:
    def __init__(self):
        self._a = A()
    
    def bar(self):
        print("B.bar()")
    
    # Показывает все методы, определенные на классе A
    def __getattr__(self, name):
        return getattr(self._a, name)
    
b = B()
b.bar() # Вызывает B.bar() (существует на B)   
b.spam(42) # Вызывает B.__getattr__('spam') и делегирует A.spam
    
    
    
    
    

B.bar()
Spam: 42


In [64]:
# Класс-прокси, который оборачивается вокруг другого объекта,
# но показывает его публичные атрибуты
class Proxy:
    def __init__(self, obj):
        self._obj = obj

    # Делегирует поиск атрибутов внутреннему obj
    def __getattr__(self, name):
        print('getattr:', name)
        return getattr(self._obj, name)

    # Делегирует присвоение атрибутов
    def __setattr__(self, name, value):
        if name.startswith('_'):
            super().__setattr__(name, value)
        else:
            print('setattr:', name, value)
            setattr(self._obj, name, value)

    # Делегирует удаление атрибутов
    def __delattr__(self, name):
        if name.startswith('_'):
            super().__delattr__(name)
        else:
            print('delattr:', name)
            delattr(self._obj, name)
            

class Spam:
    def __init__(self, x):
        self.x = x
    
    def bar(self, y):
        print('Spam.bar:', self.x, y)
        
# Создаем экземпляр
s = Spam(2)
# Создаем прокси вокруг него
p = Proxy(s)

# Access the proxy
print(p.x) 
p.bar(3) 
p.x = 37     

getattr: x
2
getattr: bar
Spam.bar: 2 3
setattr: x 37


In [67]:
class LoggedMappingMixin:
    '''Добавляет логирование для операций get/set/delete в целях отладки.'''
    __slots__ = ()
    
    def __getitem__(self, key):
        print('Getting ' + str(key))
        return super().__getitem__(key)
    
    def __setitem__(self, key, value):
        print(f'Setting {key} = {value!r}')
        return super().__setitem__(key, value)
    
    def __delitem__(self, key):
        print('Deleting ' + str(key))
        return super().__delitem__(key)
    

class SetOnceMappingMixin:
    '''Позволяет устанавливать ключ только один раз.'''
    __slots__ = ()
    
    def __setitem__(self, key, value):
        if key in self:
            raise KeyError(str(key) + ' already set')
        return super().__setitem__(key, value)
    

class StringKeysMappingMixin:
    '''Запрещает ключам быть чем-то, кроме строк.'''
    __slots__ = ()
    def __setitem__(self, key, value):
        if not isinstance(key, str):
            raise TypeError('keys must be strings')
        return super().__setitem__(key, value)
    
    
class LoggedDict(LoggedMappingMixin, dict):
    pass

d = LoggedDict()
d['x'] = 23
d['x']
del d['x']

Setting x = 23
Getting x
Deleting x


In [70]:
class RestrictKeysMixin:
    def __init__(self, *args, _restrict_key_type, **kwargs):
        self.__restrict_key_type = _restrict_key_type
        super().__init__(*args, **kwargs)

    def __setitem__(self, key, value):
        if not isinstance(key, self.__restrict_key_type):
            raise TypeError('Keys must be ' + str(self.__restrict_key_type))
        super().__setitem__(key, value)
        
class RDict(RestrictKeysMixin, dict):
    pass

d = RDict(_restrict_key_type=str)
e = RDict([('name','Dave'), ('n',37)], _restrict_key_type=str)
f = RDict(name='Dave', n=37, _restrict_key_type=str)
print(f)

{'name': 'Dave', 'n': 37}


In [6]:
class Connection:
    def __init__(self):
        self.new_state(ClosedConnectionState)
    
    def new_state(self, newstate):
        self._state = newstate
        
    # Делегирует классу состояния
    def read(self):
        return self._state.read(self)
    
    def write(self, data):
        return self._state.write(self, data)
    
    def open(self):
        return self._state.open(self)
    
    def close(self):
        return self._state.close(self)

# Базовый класс состояния соединения
class ConnectionState:
    @staticmethod
    def read(conn):
        raise NotImplementedError()
    
    @staticmethod
    def write(conn, data):
        raise NotImplementedError()
    
    @staticmethod
    def open(conn):
        raise NotImplementedError()
    
    @staticmethod
    def close(conn):
        raise NotImplementedError()
    
    
# Реализация различных состояний
class ClosedConnectionState(ConnectionState):
    @staticmethod
    def read(conn):
        raise RuntimeError('Not open')

    @staticmethod
    def write(conn, data):
        raise RuntimeError('Not open')

    @staticmethod
    def open(conn):
        conn.new_state(OpenConnectionState)

    @staticmethod
    def close(conn):
        raise RuntimeError('Already closed')
    
    
class OpenConnectionState(ConnectionState):
    @staticmethod
    def read(conn):
        print('reading')
    
    @staticmethod
    def write(conn, data):
        print('writing')
    
    @staticmethod
    def open(conn):
        raise RuntimeError('Already open')
    
    @staticmethod
    def close(conn):
        conn.new_state(ClosedConnectionState)
        

c = Connection()
print(c._state)
c.open()
print(c._state)
c.read()
c.write('hello')    

<class '__main__.ClosedConnectionState'>
<class '__main__.OpenConnectionState'>
reading
writing


In [8]:
import math
import operator

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Point({self.x!r:},{self.y!r:})'
    
    def distance(self, x, y):
        return math.hypot(self.x - x, self.y - y)
    
p = Point(2, 3)
d = getattr(p, 'distance')(0, 0) # Вызов p.distance(0, 0)
print(d)
print(operator.methodcaller('distance', 0, 0)(p))

3.605551275463989
3.605551275463989


In [10]:
points = [
    Point(1, 2),
    Point(3, 0),
    Point(10, -3),
    Point(-5, -7),
    Point(-1, 8),
    Point(3, 2)
]

# Сортируем по расстоянию от (0, 0)
points.sort(key=operator.methodcaller('distance', 0, 0))
print(points)

[Point(1,2), Point(3,0), Point(3,2), Point(-1,8), Point(-5,-7), Point(10,-3)]


In [11]:
class Node:
    pass

class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand

class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOperator):
    pass

class Sub(BinaryOperator):
    pass

class Mul(BinaryOperator):
    pass

class Div(BinaryOperator):
    pass

class Negate(UnaryOperator):
    pass

class Number(Node):
    def __init__(self, value):
        self.value = value
        
# Представление 1 + 2 * (3 - 4) / 5
t1 = Sub(Number(3), Number(4))
t2 = Mul(Number(2), t1)
t3 = Div(t2, Number(5))
t4 = Add(Number(1), t3)

In [12]:
class NodeVisitor:
    def visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)
    
    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format('visit_' + type(node).__name__))
    

class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value

    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right)

    def visit_Sub(self, node):
        return self.visit(node.left) - self.visit(node.right)

    def visit_Mul(self, node):
        return self.visit(node.left) * self.visit(node.right)

    def visit_Div(self, node):
        return self.visit(node.left) / self.visit(node.right)

    def visit_Negate(self, node):
        return -node.operand
    

e = Evaluator()
e.visit(t4)    

0.6

In [13]:
class StackCode(NodeVisitor):
    def generate_code(self, node):
        self.instructions = []
        self.visit(node)
        return self.instructions

    def visit_Number(self, node):
        self.instructions.append(('PUSH', node.value))

    def binop(self, node, instruction):
        self.visit(node.left)
        self.visit(node.right)
        self.instructions.append((instruction,))

    def visit_Add(self, node):
        self.binop(node, 'ADD')

    def visit_Sub(self, node):
        self.binop(node, 'SUB')

    def visit_Mul(self, node):
        self.binop(node, 'MUL')

    def visit_Div(self, node):
        self.binop(node, 'DIV')

    def unaryop(self, node, instruction):
        self.visit(node.operand)
        self.instructions.append((instruction,))

    def visit_Negate(self, node):
        self.unaryop(node, 'NEG')
        
        
s = StackCode()    
s.generate_code(t4)
        
        

[('PUSH', 1),
 ('PUSH', 2),
 ('PUSH', 3),
 ('PUSH', 4),
 ('SUB',),
 ('MUL',),
 ('PUSH', 5),
 ('DIV',),
 ('ADD',)]

In [19]:
import types

class Node:
    pass

class NodeVisitor:
    def visit(self, node):
        stack = [node]
        last_result = None
        while stack:
            try:
                last = stack[-1]
                if isinstance(last, types.GeneratorType):
                    stack.append(last.send(last_result))
                    last_result = None
                elif isinstance(last, Node):
                    stack.append(self._visit(stack.pop()))
                else:
                    last_result = stack.pop()
            except StopIteration:
                stack.pop()
        return last_result
    
    def _visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)

    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format('visit_' + type(node).__name__))
    

class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value

    def visit_Add(self, node):
        yield (yield node.left) + (yield node.right)

    def visit_Sub(self, node):
        yield (yield node.left) - (yield node.right)

    def visit_Mul(self, node):
        yield (yield node.left) * (yield node.right)

    def visit_Div(self, node):
        yield (yield node.left) / (yield node.right)

    def visit_Negate(self, node):
        yield - (yield node.operand)
        

class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand

class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOperator):
    pass

class Sub(BinaryOperator):
    pass

class Mul(BinaryOperator):
    pass

class Div(BinaryOperator):
    pass

class Negate(UnaryOperator):
    pass

class Number(Node):
    def __init__(self, value):
        self.value = value
        

a = Number(0)

for n in range(1,100000):
    a = Add(a, Number(n))
    
e = Evaluator()
print(e.visit(a))
    
    
    

4999950000


In [20]:
import weakref

class Node:
    def __init__(self, value):
        self.value = value
        self._parent = None
        self.children = []

    def __repr__(self):
        return 'Node({!r:})'.format(self.value)

    # Свойство, которое управляет родителем с помощью слабой ссылки
    @property
    def parent(self):
        return self._parent if self._parent is None else self._parent()

    @parent.setter
    def parent(self, node):
        self._parent = weakref.ref(node)

    def add_child(self, child):
        self.children.append(child)
        child.parent = self
        

root = Node('parent')
c1 = Node('child')
root.add_child(c1)
print(c1.parent)
del root
print(c1.parent)

Node('parent')
None


In [25]:
# Класс, созданный, чтобы проиллюстрировать то, что будет при удалении
class Data:
    def __del__(self):
        print('Data.__del__')

# Класс-узел, в котором есть цикл
class Node:
    def __init__(self):
        self.data = Data()
        self.parent = None
        self.children = []
        
    def add_child(self, child):
        self.children.append(child)
        child.parent = self
        
a = Data()
del a # Сразу же удаляется
a = Node()
del a # Сразу же удаляется
a = Node()
a.add_child(Node())
del a # Не удаляется (и сообщение не выводится)

Data.__del__
Data.__del__


In [28]:
from functools import total_ordering

class Room:
    def __init__(self, name, length, width):
        self.name = name
        self.length = length
        self.width = width
        self.square_feet = self.length * self.width

@total_ordering
class House:
    def __init__(self, name, style):
        self.name = name
        self.style = style
        self.rooms = list()

    @property
    def living_space_footage(self):
        return sum(r.square_feet for r in self.rooms)

    def add_room(self, room):
        self.rooms.append(room)

    def __str__(self):
        return f'{self.name}: {self.living_space_footage} square foot {self.style}'

    def __eq__(self, other):
        return self.living_space_footage == other.living_space_footage
    
    def __lt__(self, other):
        return self.living_space_footage < other.living_space_footage
    
    
# Построим несколько домов и добавим в них комнаты
h1 = House('h1', 'Cape')
h1.add_room(Room('Master Bedroom', 14, 21))
h1.add_room(Room('Living Room', 18, 20))
h1.add_room(Room('Kitchen', 12, 16))
h1.add_room(Room('Office', 12, 12))

h2 = House('h2', 'Ranch')
h2.add_room(Room('Master Bedroom', 14, 21))
h2.add_room(Room('Living Room', 18, 20))
h2.add_room(Room('Kitchen', 12, 16))

h3 = House('h3', 'Split')
h3.add_room(Room('Master Bedroom', 14, 21))
h3.add_room(Room('Living Room', 18, 20))
h3.add_room(Room('Office', 12, 16))
h3.add_room(Room('Kitchen', 15, 17))

houses = [h1, h2, h3]

print('Is h1 bigger than h2?', h1 > h2)
print('Is h2 smaller than h3?', h2 < h3)
print('Is h2 greater than or equal to h1?', h2 >= h1)
print('Which one is biggest?', max(houses))
print('Which is smallest?', min(houses))

Is h1 bigger than h2? True
Is h2 smaller than h3? True
Is h2 greater than or equal to h1? False
Which one is biggest? h3: 1101 square foot Split
Which is smallest? h2: 846 square foot Ranch


In [32]:
import logging

a = logging.getLogger('foo')
b = logging.getLogger('bar')
print(a is b)
c = logging.getLogger('foo')
print(a is c)

# Опрашиваемый класс
class Spam:
    def __init__(self, name):
        self.name = name
        
# Поддержка кеширования
_spam_cache = weakref.WeakValueDictionary()

def get_spam(name):
    if name not in _spam_cache:
        s = Spam(name)
        _spam_cache[name] = s
    else:
        s = _spam_cache[name]
    return s

a = get_spam('foo')
b = get_spam('bar')
print(a is b)
c = get_spam('foo')
print(a is c)

False
True
False
True


In [33]:
class CachedSpamManager:
    def __init__(self):
        self._cache = weakref.WeakValueDictionary()
    
    def get_spam(self, name):
        if name not in self._cache:
            s = Spam(name)
            self._cache[name] = s
        else:
            s = self._cache[name]
        return s
    
    def clear(self):
        self._cache.clear()

class Spam:
    manager = CachedSpamManager()
    def __init__(self, name):
        self.name = name
    
    def get_spam(name):
        return Spam.manager.get_spam(name)
    
a = Spam('foo')
b = Spam('foo')
a is b

False